##### Copyright 2025 Google LLC.

# Gemini API: Analyze a Video - Historic Event Recognition

This notebook shows how you can use Gemini models' multimodal capabilities to recognize which historic event is happening in the video.

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Analyze_a_Video_Historic_Event_Recognition.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In [1]:
%pip install -U -q "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 8.4 MB/s eta 0:00:00


## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [17]:
from google import genai
from google.colab import userdata

API_KEY = userdata.get('google_api_key')
client = genai.Client(api_key=API_KEY)

## Example

This example uses [video of President Ronald Reagan's Speech at the Berlin Wall](https://s3.amazonaws.com/NARAprodstorage/opastorage/live/16/147/6014716/content/presidential-libraries/reagan/5730544/6-12-1987-439.mp4) taken on June 12 1987.

In [8]:
# Download video
path = "berlin.mp4"
url = "https://s3.amazonaws.com/NARAprodstorage/opastorage/live/16/147/6014716/content/presidential-libraries/reagan/5730544/6-12-1987-439.mp4"
!wget $url -O $path

--2025-08-22 08:55:35--  https://s3.amazonaws.com/NARAprodstorage/opastorage/live/16/147/6014716/content/presidential-libraries/reagan/5730544/6-12-1987-439.mp4
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.117.224, 52.216.184.213, 16.182.100.16, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.117.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628645171 (600M) [video/mp4]
Saving to: ‘berlin.mp4’

berlin.mp4          100%[===================>] 599.52M  34.9MB/s    in 16s     

2025-08-22 08:55:52 (36.8 MB/s) - ‘berlin.mp4’ saved [628645171/628645171]



In [18]:
# Upload video
video_file = client.files.upload(file=path)

In [19]:
import time
# Wait until the uploaded video is available
while video_file.state.name == "PROCESSING":
  print('.', end='')
  time.sleep(5)
  video_file = client.files.get(name=video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

.......................

The uploaded video is ready for processing. This prompt instructs the model to provide basic information about the historical events portrayed in the video.

In [20]:
system_prompt = """
  You are historian who specializes in events caught on film.
  When you receive a video answer following questions:
  When did it happen?
  Who is the most important person in video?
  How the event is called?
"""

Some historic events touch on controversial topics that may get flagged by Gemini API, which blocks the response for the query.

Because of this, it might be a good idea to turn off safety settings.

In [21]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [23]:
from google.genai import types

MODEL_ID="gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true, isTemplate: true}
response = client.models.generate_content(
    model=f"models/{MODEL_ID}",
    contents=[
        "what is the video about and how many persons are present in the video",
        video_file
        ],
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        safety_settings=safety_settings,
        ),
    )
print(response.text)

This video is about **Ronald Reagan's "Tear Down This Wall" speech** delivered at the Brandenburg Gate in West Berlin, Germany.

*   **When did it happen?** The speech took place on **June 12, 1987**. Reagan references President Kennedy's visit 24 years prior (1963) and the 40th anniversary of the Marshall Plan (1947), both of which align with 1987.
*   **Who is the most important person in the video?** The most important person is **Ronald Reagan**, the then-President of the United States, who delivered the iconic speech.
*   **How the event is called?** It is widely known as the **"Tear Down This Wall" speech** or **Ronald Reagan's Brandenburg Gate speech**.

Regarding the number of persons present in the video:
There are approximately **18-20 dignitaries** visible on the stage with President Reagan. The crowd in front of the stage consists of **hundreds, if not thousands, of people**.


As you can see, the model correctly provided information about the dates, Ronald Reagan, who was the main subject of the video, and the name of this event.

You can delete the video to prevent unnecessary data storage.

In [ ]:
# Delete video
client.files.delete(name=video_file.name)

DeleteFileResponse()

## Summary

Now you know how you can prompt Gemini models with videos and use them to recognize historic events.

This notebook shows only one of many use cases. Check the [Video understanding](../quickstarts/Video_understanding.ipynb) notebook for more examples of using the Gemini API with videos.